<a href="https://www.kaggle.com/code/shruthimohan/transcribing-a-video-with-openai-whisper-working?scriptVersionId=218770955" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Getting the audio from a youtube video

In [2]:
pip install yt-dlp pydub SpeechRecognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.1/172.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 51.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 54.8 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import re
from yt_dlp import YoutubeDL
import speech_recognition as sr

def download_audio_from_youtube(youtube_url):
    options = {
        'format': 'bestaudio/best',
        'outtmpl': '%(title)s.%(ext)s',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
            'preferredquality': '192',
        }],
    }
    with YoutubeDL(options) as ydl:
        info = ydl.extract_info(youtube_url, download=True)
        downloaded_files = os.listdir(os.getcwd())
        for file in downloaded_files:
            if file.endswith('.wav'):
                print(f"Downloaded and converted audio to WAV: {file}")
                return file
    
    raise FileNotFoundError(f"Could not find the expected audio file: {original_file}")

### Transcribe the audio

In [1]:
!sudo apt install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 132 not upgraded.


In [4]:
pip install openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 14.2 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803373 sha256=442711b5e721bc443b33560256367be640d367a46b0fbd9282430e6e74ccee65
  Stored in directory: /root/.cache/pip/wheels/dd/4a/1f/d1c4bf3b9133c8168fe617ed979cab7b14fe381d059ffb9d83
Successfully built openai-whisper
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
Note: you may need to restart the kernel to use updated packages.


In [13]:
import whisper
def transcribe_audio(audio_file_path):
    # Load the Whisper model
    model = whisper.load_model("base")  # Use "base" for speed, or "small", "medium", "large" for accuracy

    print('Transcribing...')
    # Transcribe the audio
    result = model.transcribe(audio_file_path)
    print('Transcription is ready')
    return result["text"]

### Run the workflow using a video

In [14]:
if __name__ == "__main__":
    youtube_url = "https://www.youtube.com/watch?v=7ok49o24SDw"
    
    try:
        # Step 1: Download audio from YouTube
        wav_file = download_audio_from_youtube(youtube_url)  

        # Step 2: Transcribe audio
        transcription = transcribe_audio(wav_file)
        print("\nTranscription:\n")
        print(transcription)
        
        # Step 3: Save transcription to a file
        filename = "transcribed_text1.txt"
        with open(filename, "w") as file:
            file.write(transcription)
        print(f"Transcription has been saved to {filename}")
    except Exception as e:
        print(f"An error occurred: {e}")

[youtube] Extracting URL: https://www.youtube.com/watch?v=7ok49o24SDw
[youtube] 7ok49o24SDw: Downloading webpage
[youtube] 7ok49o24SDw: Downloading ios player API JSON
[youtube] 7ok49o24SDw: Downloading mweb player API JSON
[youtube] 7ok49o24SDw: Downloading m3u8 information
[info] 7ok49o24SDw: Downloading 1 format(s): 251
[download] Lecture 5： Text Processing： Basics.webm has already been downloaded
[download] 100% of   26.63MiB
[ExtractAudio] Destination: Lecture 5： Text Processing： Basics.wav
Deleting original file Lecture 5： Text Processing： Basics.webm (pass -k to keep)
Downloaded and converted audio to WAV: Lecture 5： Text Processing： Basics.wav


/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


Transcribing...
Transcription is ready

Transcription:

 Hello everyone, welcome back to the final lecture of the first week. So, in the last lecture we were discussing about various empirical laws. So, in particular, GIFs law and Heaps law that how the words in the vocabulary are distributed in a corpus. And we saw that the distribution is not very uniform. There are certain words that are very, very common. So, we saw that roughly 100 words in the vocabulary make for 50 percent of the corpus, but by that I mean that number of tokens. And on the other hand, there are 50 percent of the words in the vocabulary that occur only once. And we discussed what are the various relationships among the vocabulary size and the number of tokens that I observe in a corpus. And also how they grow with us to each other. And GIFs law gives gave me a relation between the frequency and the rank of a word. So, today in this lecture we will start with the basic key processing in language. So, we will cover